In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão de Python Neste Jupyter Notebook:', python_version())

# usaremos o filtro 'waning' para deixar mais limpo.
import warnings
warnings.filterwarnings('ignore')

### Representing Search Problems

In [ ]:
# scripts/searchProblem.py
class Search_problem(object):
    """
    A search problem consists of:
    * a start node
    * a neighbors function that gives the neighbors of a node
    * a specification of a goal
    * a (optional) heuristic function.
    The methods must be overridden to define a search problem.
    """

    def start_node(self):
        """returns start node"""
    
        raise NotImplementedError("start_node") # abstract method

    def is_goal(self,node):
        """is True if node is a goal"""

        raise NotImplementedError("is_goal") # abstract method

    def neighbors(self,node):
        """returns a list of the arcs for the neighbors of node"""

        raise NotImplementedError("neighbors") # abstract method

    def heuristic(self,n):
        """
        Gives the heuristic value of node n.
        Returns 0 if not overridden.
        """
        
        return 0

In [ ]:
# scripts/searchProblem.py
class Arc(object):
    """
    An arc has a from_node and a to_node node and a (non-negative)
    cost
    """
    
    def __init__(self, from_node, to_node, cost=1, action=None):
    
        assert cost >= 0, ("Cost cannot be negative for" + str(from_node) + "->" + str(to_node) + ", cost: " + str(cost))
        self.from_node = from_node
        self.to_node = to_node
        self.action = action
        self.cost = cost

    def __repr__(self):
        """
        string representation of an arc
        """
        
        if self.action:
            return str(self.from_node) + " --" + str(self.action) + "-->" + str(self.to_node)
        
        else:
            return str(self.from_node) + " --> " + str(self.to_node)

In [ ]:
# scripts/searchProblem.py
class Search_problem_from_explicit_graph(Search_problem):
    """
    A search problem consists of:
    * a list or set of nodes
    * a list or set of arcs
    * a start node
    * a list or set of goal nodes
    * a dictionary that maps each node into its heuristic value.
    * a dictionary that maps each node into its (x,y) position
    """

    def __init__(self, nodes, 
                 arcs, start = None,
                 goals = set(), hmap = {},
                 positions = {}):
        
        self.neighs = {}
        self.nodes = nodes
    
        for node in nodes:
            self.neighs[node]=[]
            self.arcs = arcs
    
        for arc in arcs:
            self.neighs[arc.from_node].append(arc)
            self.start = start
            self.goals = goals
            self.hmap = hmap
            self.positions = positions


    def start_node(self):
        """
        returns start node
        """
        
        return self.start


    def is_goal(self,node):
        """
        is True if node is a goal
        """
        
        return node in self.goals


    def neighbors(self,node):
        """
        returns the neighbors of node
        """
    
        return self.neighs[node]
    
    
    def heuristic(self,node):
        """
        Gives the heuristic value of node n.
        Returns 0 if not overridden in the hmap.
        """
        
        if node in self.hmap:
            return self.hmap[node]
        
        else:
            return 0

    def __repr__(self):
        """
        returns a string representation of the search problem
        """
        
        res=""
        for arc in self.arcs:
            res += str(arc)+". "
        
        return res

In [ ]:
class Path(object):
    """
    A path is either a node or a path followed by an arc
    """

    
    def __init__(self,initial,arc=None):
        """
        initial is either a node (in which case arc is None) or
        a path (in which case arc is an object of type Arc)
        """
        
        self.initial = initial
        self.arc=arc
        
        if arc is None:
            self.cost=0
        
        else:
            self.cost = initial.cost + arc.cost

    def end(self):
        """
        returns the node at the end of the path
        """
        
        if self.arc is None:
            return self.initial
        else:
            return self.arc.to_node


    def nodes(self):
        """
        enumerates the nodes for the path.
        This starts at the end and enumerates nodes in the path
        backwards.
        """
        
        current = self
        
        while current.arc is not None:
            yield current.arc.to_node
            current = current.initial
        yield current.initial

    def initial_nodes(self):
        """
        enumerates the nodes for the path before the end node.
        This starts at the end and enumerates nodes in the path
        backwards.
        """
        
        if self.arc is not None:
            yield from self.initial.nodes()

    def __repr__(self):
        """
        returns a string representation of a path
        """
        
        if self.arc is None:
            return str(self.initial)
        
        elif self.arc.action:
            return (str(self.initial) + "\n --" + str(self.arc.action) + "--> " + str(self.arc.to_node))
    
        else:
            return str(self.initial) + " --> " + str(self.arc.to_node)

In [ ]:
problem1 = Search_problem_from_explicit_graph({'A','B','C','D','G'},
                                              [Arc('A','B',3),
                                               Arc('A','C',1),
                                               Arc('B','D',1),
                                               Arc('B','G',3),
                                               Arc('C','B',1),
                                               Arc('C','D',3),
                                               Arc('D','G',1)],
                                              
                                              start = 'A',
                                              goals = {'G'},
                                              positions={'A': (0, 2), 
                                                         'B': (1, 1),
                                                         'C': (0, 1),
                                                         'D': (1, 0),
                                                         'G':(2, 0)})

In [ ]:
problem2 = Search_problem_from_explicit_graph({'a','b','c','d','e','g','h','j'},
                                              [Arc('a','b',1),
                                               Arc('b','c',3), 
                                               Arc('b','d',1), 
                                               Arc('d','e',3),
                                               Arc('d','g',1),
                                               Arc('a','h',3),
                                               Arc('h','j',1)],
                                              
                                              start = 'a',
                                              goals = {'g'},
                                              positions={'a': (0, 0),
                                                         'b': (0, 1), 
                                                         'c': (0, 4),
                                                         'd': (1, 1),
                                                         'e': (1, 4),
                                                         'g': (2, 1),
                                                         'h': (3, 0),
                                                         'j': (3, 1)})

In [ ]:
problem3 = Search_problem_from_explicit_graph({'a','b','c','d','e','g','h','j'},
                                              [],
                                              start = 'g',
                                              goals = {'k','g'})

In [ ]:
simp_delivery_graph = Search_problem_from_explicit_graph({'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'},
                                                         [Arc('A', 'B', 2), Arc('A', 'C', 3),
                                                          Arc('A', 'D', 4), Arc('B', 'E', 2),
                                                          Arc('B', 'F', 3), Arc('C', 'J', 7),
                                                          Arc('D', 'H', 4), Arc('F', 'D', 2),
                                                          Arc('H', 'G', 3), Arc('J', 'G', 4)],
                                                         
                                                         start = 'A',
                                                         goals = {'G'},
                                                         hmap = {'A': 7, 'B': 5,
                                                                 'C': 9, 'D': 6,
                                                                 'E': 3, 'F': 5,
                                                                 'G': 0, 'H': 3,
                                                                 'J': 4,})

In [ ]:
cyclic_simp_delivery_graph = Search_problem_from_explicit_graph({'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J'},
                                                                [Arc('A', 'B', 2), Arc('A', 'C', 3),
                                                                 Arc('A', 'D', 4), Arc('B', 'A', 2),
                                                                 Arc('B', 'E', 2), Arc('B', 'F', 3),
                                                                 Arc('C', 'A', 3), Arc('C', 'J', 7),
                                                                 Arc('D', 'A', 4), Arc('D', 'H', 4),
                                                                 Arc('F', 'B', 3), Arc('F', 'D', 2),
                                                                 Arc('G', 'H', 3), Arc('G', 'J', 4),
                                                                 Arc('H', 'D', 4), Arc('H', 'G', 3),
                                                                 Arc('J', 'C', 6), Arc('J', 'G', 4)]
                                                                
                                                                ,start = 'A',
                                                                goals = {'G'},
                                                                hmap = {'A': 7, 'B': 5,
                                                                        'C': 9, 'D': 6,
                                                                        'E': 3, 'F': 5,
                                                                        'G': 0, 'H': 3,
                                                                        'J': 4,})

In [ ]:
acyclic_delivery_problem = Search_problem_from_explicit_graph({'mail', 'ts', 'o103', 'o109', 'o111',
                                                               'b1', 'b2', 'b3', 'b4','c1', 'c2', 'c3',
                                                               'o125', 'o123', 'o119', 'r123', 'storage'},
                                                              [Arc('ts','mail',6), Arc('o103','ts',8),
                                                               Arc('o103','b3',4), Arc('o103','o109',12),
                                                               Arc('o109','o119',16), Arc('o109','o111',4),
                                                               Arc('b1','c2',3), Arc('b1','b2',6),
                                                               Arc('b2','b4',3), Arc('b3','b1',4),
                                                               Arc('b3','b4',7), Arc('b4','o109',7),
                                                               Arc('c1','c3',8), Arc('c2','c3',6),
                                                               Arc('c2','c1',4), Arc('o123','o125',4),
                                                               Arc('o123','r123',4), Arc('o119','o123',9),
                                                               Arc('o119','storage',7)],
                                                              start = 'o103',
                                                              goals = {'r123'},
                                                              hmap = {'mail' : 26,
                                                                      'ts' : 23,
                                                                      'o103' : 21,
                                                                      'o109' : 24,
                                                                      'o111' : 27,
                                                                      'o119' : 11,
                                                                      'o123' : 4,
                                                                      'o125' : 6,
                                                                      'r123' : 0,
                                                                      'b1' : 13,
                                                                      'b2' : 15,
                                                                      'b3' : 17,
                                                                      'b4' : 18,
                                                                      'c1' : 6,
                                                                      'c2' : 10,
                                                                      'c3' : 12,
                                                                      'storage' : 12
                                                                      })

In [ ]:
cyclic_delivery_problem = Search_problem_from_explicit_graph({'mail', 'ts', 'o103', 'o109', 'o111',
                                                              'b1', 'b2', 'b3', 'b4', 'c1', 'c2', 'c3',
                                                              'o125','o123','o119','r123','storage'},
                                                             [Arc('ts','mail',6), Arc('mail','ts',6), 
                                                              Arc('o103','ts',8), Arc('ts','o103',8),
                                                              Arc('o103','b3',4), Arc('o103','o109',12),
                                                              Arc('o109','o103',12), Arc('o109','o119',16), 
                                                              Arc('o119','o109',16), Arc('o109','o111',4),
                                                              Arc('o111','o109',4), Arc('b1','c2',3),
                                                              Arc('b1','b2',6), Arc('b2','b1',6),
                                                              Arc('b2','b4',3), Arc('b4','b2',3),
                                                              Arc('b3','b1',4), Arc('b1','b3',4),
                                                              Arc('b3','b4',7), Arc('b4','b3',7),
                                                              Arc('b4','o109',7), Arc('c1','c3',8), 
                                                              Arc('c3','c1',8), Arc('c2','c3',6),
                                                              Arc('c3','c2',6), Arc('c2','c1',4),
                                                              Arc('c1','c2',4), Arc('o123','o125',4), 
                                                              Arc('o125','o123',4), Arc('o123','r123',4), 
                                                              Arc('r123','o123',4), Arc('o119','o123',9),
                                                              Arc('o123','o119',9), Arc('o119','storage',7),
                                                              Arc('storage','o119',7)],
                                                             start = 'o103',
                                                             goals = {'r123'},
                                                             hmap = {'mail' : 26,
                                                                     'ts' : 23,
                                                                     'o103' : 21,
                                                                     'o109' : 24,
                                                                     'o111' : 27,
                                                                     'o119' : 11,
                                                                     'o123' : 4,
                                                                     'o125' : 6,
                                                                     'r123' : 0,
                                                                     'b1' : 13,
                                                                     'b2' : 15,
                                                                     'b3' : 17,
                                                                     'b4' : 18,
                                                                     'c1' : 6,
                                                                     'c2' : 10,
                                                                     'c3' : 12,
                                                                     'storage' : 12
                                                                     })

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

### End.